In [ ]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="nomic-embed-text"
)


Imported tools from: /Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/tools.py
CWD: /Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk
First sys.path entries:
- /opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python313.zip
- /opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13
- /opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload
- 
- /Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/venv/lib/python3.13/site-packages
Has TOOLS? False


In [ ]:
from pydantic import BaseModel

class CalcArgs(BaseModel):
    expr: str

# -------- 1) Tools (deterministic) --------
def calculator(args: CalcArgs) -> str:
    return str(eval(args.expr, {"__builtins__": {}}, {}))

def factorial(n: int) -> str:
    if n < 0:
        return "Error: factorial undefined for negative numbers."
    result = 1
    for i in range(2, n+1):
        result *= i
    return str(result)\
    
def word_count(text: str) -> str:
    return str(len(text.split()))

def reverse(text: str) -> str:
    return text[::-1]

def uppercase(text: str) -> str:
    return text.upper()


TOOLS = {
    "calculator": {
        "fn": calculator,
        "schema": {"expr": str},
        "desc": "Evaluate a math expression."
    },
    "word_count": {
        "fn": word_count,
        "schema": {"text": str},
        "desc": "Count words in text."
    },
    "reverse": {
        "fn": reverse,
        "schema": {"text": str},
        "desc": "Reverse the text."
    },
    "uppercase": {
        "fn": uppercase,
        "schema": {"text": str},
        "desc": "Convert text to uppercase."
    },
    "factorial": {
        "fn": factorial,
        "schema": {"n": int},
        "desc": "Return the factorial of a non-negative integer n."
    }
}

In [33]:
import requests, json, re
from typing import Callable, Dict, Any

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL = "llama3"

# -------- 2) System prompts (strict & deterministic) --------
tool_list = '" | "'.join(TOOLS.keys())
ROUTER_SYSTEM = f"""You are a strict router. Output ONLY one JSON object, no prose.
Schema:
{{
  "decision": "tool" | "answer",
  "tool_name": null | "{tool_list}",
  "arguments": null | {
      "expr": "string"    
      "text": "string"    
      "n": int    
  },        
  "reason": "short",
  "confidence": 0.9
}}
Rules:
- If the query requires math, output calculator with {"expr":"..."}.
- If the query asks about number of words, output word_count with {"text":"..."}.
- If the query asks to reverse or uppercase text, use those tools.
- If the query is factorial of N, use factorial with {"n":N}.
- Otherwise answer directly with "decision":"answer".
Output exactly one line of JSON, nothing else.
"""

ANSWER_SYSTEM = """You are a concise, deterministic assistant. 
Use only the provided tool result if present. No randomness, no hedging, no extra synonyms. 
Respond with a single short paragraph or a single value, whichever is appropriate.
"""

# -------- 3) Ollama helper (deterministic settings) --------
def chat(messages):
    r = requests.post(
        OLLAMA_URL,
        json={
            "model": MODEL,
            "messages": messages,
            "stream": False,
            "options": {
                "temperature": 0.0,  # deterministic
                "top_p": 1.0,
                "top_k": 1,
                "repeat_penalty": 1.1,
                "seed": 12345,       # same input -> same output
            },
        },
        timeout=120,
    )
    r.raise_for_status()
    return r.json()["message"]["content"]

# -------- 4) Minimal JSON validator/dispatcher --------
def validate_and_run(tool_name: str, arguments: dict) -> str:
    if tool_name not in TOOLS:
        raise ValueError(f"Unknown tool: {tool_name}")
    spec = TOOLS[tool_name]
    # Minimal schema/type check
    for key, typ in spec["schema"].items():
        if key not in arguments or not isinstance(arguments[key], typ):
            raise ValueError(f"Invalid arguments for {tool_name}: expected {key}:{typ.__name__}")
    return str(spec["fn"](**arguments))

# -------- 5) Public entry: ask(query) --------
def ask(query: str) -> str:
    # Step A: Route (decide tool vs direct answer)
    router_msgs = [
        {"role": "system", "content": ROUTER_SYSTEM},
        {"role": "user", "content": query},
    ]
    decision_text = chat(router_msgs).strip()

    # Robust JSON parse (one-line object only)
    try:
        decision = json.loads(decision_text)
    except json.JSONDecodeError as e:
        # If model somehow drifted, fall back to direct answering deterministically
        answer_msgs = [
            {"role": "system", "content": ANSWER_SYSTEM},
            {"role": "user", "content": query},
        ]
        return chat(answer_msgs)

    if decision.get("decision") == "tool":
        tool_name = decision.get("tool_name")
        arguments = decision.get("arguments") or {}
        # Execute tool deterministically
        tool_out = validate_and_run(tool_name, arguments)

        # Step B: Finalize answer (use tool result only)
        final_msgs = [
            {"role": "system", "content": ANSWER_SYSTEM},
            {"role": "user", "content": query},
            {"role": "assistant", "content": json.dumps(decision, separators=(",", ":"))},  # echo the tool call
            {"role": "tool", "content": tool_out},
            {"role": "user", "content": "Use the tool result above and answer with a single, deterministic response."},
        ]
        return chat(final_msgs)
    else:
        # Direct deterministic answer (no tool)
        answer_msgs = [
            {"role": "system", "content": ANSWER_SYSTEM},
            {"role": "user", "content": query},
        ]
        return chat(answer_msgs)

# -------- 6) Demo --------
if __name__ == "__main__":
    print("Q1:", "(2+3)*7 via calculator")
    print("A1:", ask("What is (2+3)*7? Use the calculator."))
    print("Q2:", "word count")
    print("A2:", ask("How many words are in: 'The quick brown fox jumps over the lazy dog'?"))
    print("Q3:", "Reverse")
    print("A3:", ask("Reverse this text exactly: hello world"))
    print("Q4:", "Uppercase")
    print("A4:", ask("Make this uppercase exactly: Satellite link test"))
    print("Q5:", "Factorial of 5")
    print("A5:", ask("What is the factorial of 5"))


ValueError: Invalid format specifier ' "string"    
      "text": "string"    
      "n": int    
  ' for object of type 'str'